This code possibly to be utilized later

In [40]:
# from fasta alignment
import collections
import utils

deletes_number = 0

dataframes = [
    (fasta_unaligned_df,   '../../data/csv/fasta.csv'),
    (fasta_unaligned_L_df,   '../../data/csv/fasta_L.csv'),
    (fasta_unaligned_H_df,   '../../data/csv/fasta_H.csv'),
    (sasa_unaligned_df,   '../../data/csv/sasa_merged_relative.csv'),
    (fasta_aligned_L_df,   '../../data/csv/fasta_aho_L.csv'),
    (fasta_aligned_H_df,   '../../data/csv/fasta_aho_H.csv'),
    (fasta_aligned_df,   '../../data/csv/fasta_aho.csv'),
]

fasta_unaligned_lengths = utils.nondash_counts(fasta_unaligned_df)
uneven_length_removals = collections.defaultdict(list)
for sasa_index, sasa_row in sasa_unaligned_df.iterrows():
    fasta_row = fasta_unaligned_df.loc[sasa_index]
    
    sasa_len = sasa_row.count()
    fasta_len = fasta_unaligned_lengths[sasa_index]
    len_diff = abs(sasa_len - fasta_len)
    
    if len_diff > 0:
        deletes_number += 1
        if deletes_number > 20: break
        uneven_length_removals['index'].append(sasa_index)
        uneven_length_removals['fasta_len'].append(fasta_len)
        uneven_length_removals['sasa_len'].append(sasa_len)
        uneven_length_removals['len_diff'].append(len_diff)
        
        # uncomment this block later
        for df, path in dataframes:
            df.drop(sasa_index, inplace=True, errors='ignore')

for df, path in dataframes:
    df.to_csv(path)


uneven_length_removals_index = uneven_length_removals['index']
if uneven_length_removals_index:
    print('generating new removal data')
    del uneven_length_removals['index']
    uneven_length_removals_df = pd.DataFrame(uneven_length_removals, index=uneven_length_removals_index)
    uneven_length_removals_df.to_csv('../../data/csv/uneven_length_removals.csv')
else:
    # This cell was executed already on this dataset
    #   so the list of uneven lengths will be empty
    # It is stored on the disk already, load it from there
    print('nothing new there, loading the data from the disk')
    uneven_length_removals_df = pd.read_csv('../../data/csv/uneven_length_removals.csv', index_col=0)

uneven_length_removals_df

nothing new there, loading the data from the disk


,fasta_len,sasa_len,len_diff
2QTJ:H,475,400,75
3CHN:H,475,133,342
3CHN:L,214,66,148
3CM9:H,462,144,318
3CM9:L,214,61,153
3HC4:L,213,212,1
3M8O:H,221,219,2
3QHZ:L,216,215,1
4P3D:H,218,217,1
5T5B:L,210,209,1


In [ ]:
# remove those sequences that do not have any cluster assigned
keys_to_remove = X[~X['Id'].isin(clusters_df['sequence_id'])].index.to_list()
X.drop(index=keys_to_remove, inplace=True)
Y.drop(index=keys_to_remove, inplace=True)
print(f'keys removed {keys_to_remove}')

In [10]:
train_keys = set(X['Id']).intersection(set(clustering_dataset_ids))
test_keys = set(X['Id']).intersection(set(test_ids))
X_train, X_test = X[X['Id'].isin(train_keys)], X[X['Id'].isin(test_keys)]
Y_train, Y_test = Y[Y['Id'].isin(train_keys)], Y[Y['Id'].isin(test_keys)]
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((3801, 156), (435, 156), (3801, 156), (435, 156))

In [ ]:
clusters_df = pd.read_csv(CLUSTER_FILE, index_col='sequence_id').drop(columns='Unnamed: 0')
clusters_df['Id'] = clusters_df.index
clusters_df['cluster'] = clusters_df['cluster']
X_train = pd.read_csv('../../data/csv/x_train.csv', index_col=0)
Y_train = pd.read_csv('../../data/csv/y_train.csv', index_col=0)
m = X_train.merge(clusters_df, on='Id', how='inner')
groups = pd.Series(m['cluster'].values, index=m['Id'])

X_train.shape, Y_train.shape

In [18]:
def create_miniset(X: pd.DataFrame, Y: pd.DataFrame, clust_order_start: int, clust_order_stop: int) -> tuple:
    miniset_clusters = np.array(clusters_df['cluster'].value_counts()[clust_order_start:clust_order_stop].index)
    miniset_ids = clusters_df[clusters_df['cluster'].isin(miniset_clusters)].index
    miniset_ids = list(set(miniset_ids).intersection(set(X.index)))
    Xm, Ym = X.loc[miniset_ids, :], Y.loc[miniset_ids, :]
    return get_oh_matrix(Xm), Ym

In [ ]:
@ignore_warnings(category=ConvergenceWarning)
def test_model(clf, input_params, xtr, ytr, xte, yte):    
    pipeline, params, name = clf(input_params)
    mor = MultiOutputRegressor(estimator=pipeline)
    mor.fit(xtr, ytr)
    predicts = mor.predict(xte)
    
    result_dev = utils.avg_deviation(yte, predicts)
    print(f'{result_dev:.2f}%' , end='')
    return result_dev, predicts

In [ ]:
def run(clf, Xmtr, Ymtr, Xmte, Ymte):
    print(f'starting Xtrain: {Xmtr.shape} Xtest: {Xmte.shape}')
    max_iter_values = [1, 5, 10, 20, 50, 100, 500, 1000, 5000]
    for max_iter in max_iter_values:   
        print(f'max_iter: {max_iter} - ', end='')
        runtime = t.timeit(lambda: test_model(clf, max_iter, 
                                              Xmtr, Ymtr, Xmte, Ymte), number=1)
        print(f' - {runtime:.2f}s')

In [ ]:
def svm(n=-1):        
    ppl = Pipeline([
        ('scaler', StandardScaler()),
        #('kernel_approx', Nystroem()),
        ('svm', SVR(max_iter=n) if n>-1 else SVR())
    ])
    parameters = {
        'estimator__svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'estimator__svm__max_iter': [1, 10, 50, 200, 500, 1000],
        'estimator__svm__C' : np.logspace(0.01, 3, num=30),
        'estimator__svm__degree' : [1, 3, 8],
        'estimator__svm__coef0' : [0.01,10,0.5],
        'estimator__svm__gamma' : ('auto','scale'),
        #'estimator__kernel_approx__kernel': ['linear'],
        #'estimator__kernel_approx__gamma': [1],
        #'estimator__kernel_approx__n_components': [30],
    }
    return ppl, parameters, 'SVM'

In [ ]:
run(svm, Xmtr, Ymtr, Xmte, Ymte)

In [44]:
def elastic_net(n=-1):
    ppl = Pipeline([
        ('scaler', StandardScaler()),
        ('en', ElasticNet(max_iter=n) if n>-1 else ElasticNet())
    ])
    parameters = {
        'estimator__en__alpha' : np.logspace(0.01, 3, num=30),
        'estimator__en__positive': [True, False],
        'estimator__en__tol': [0.0001, 0.001, 0.00001],
        'estimator__en__max_iter': [1, 10, 50, 200, 500, 1000],
    }
    return ppl, parameters, 'ElasticNet'

In [45]:
def lasso(n=-1):
    ppl = Pipeline([
        ('scaler', StandardScaler()),
        ('lasso', Lasso(max_iter=n) if n>-1 else Lasso())
    ])
    parameters = {
        'estimator__lasso__precompute': [True],
        'estimator__lasso__alpha' : np.logspace(0.01, 3, num=30),
        'estimator__lasso__positive': [True, False],
        'estimator__lasso__tol': [0.0001, 0.001, 0.00001],
        'estimator__lasso__max_iter': [1, 10, 50, 200, 500, 1000],
    }
    return ppl, parameters, 'Lasso'

In [46]:
def kernel_ridge(n=-1):
    ppl = Pipeline([
        ('scaler', StandardScaler()),
        ('kr', KernelRidge())
    ])
    parameters = {
        #'estimator__kr__kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),
        #'estimator__kr__alpha' : np.logspace(0.01, 3, num=30),
        #'estimator__kr__degree' : [1, 2, 4, 8],
        #'estimator__kr__coef0' : np.linspace(0.001, 50, num=10),
        #'estimator__kr__gamma' : (1, 5, 10, 50, 100, 0.5),
    }
    return ppl, parameters, 'KernelRidge'

In [47]:
def gradient_boosting(n=-1):
    ppl = Pipeline([
        ('scaler', StandardScaler()),
        ('gbr', GradientBoostingRegressor(n_estimators=n) if n>-1 else GradientBoostingRegressor())
    ])
    parameters = {
        'estimator__gbr__loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
        'estimator__gbr__n_estimators': [1, 2, 5],
        'estimator__gbr__criterion': ['friedman_mse', 'squared_error', 'mse', 'mae']
    }
    return ppl, parameters, 'GradientBoosting'

In [ ]:
# TODO complete the list
models = [svm, elastic_net, lasso, gradient_boosting]
loss_function = make_scorer(utils.avg_deviation, greater_is_better=False)

In [100]:
results = dict()
#for model in tqdm(models, total=len(models)):
#for model in models:

logger.info(model)
pipeline, params, name = model()
mor = MultiOutputRegressor(estimator=pipeline)

splitter = LeaveOneGroupOut()
split = splitter.split(Xmtr, Ymtr, groups=groups)
#grid = RandomizedSearchCV(mor, {}, verbose=0, 
#                          n_iter=1, n_jobs=-1, scoring=loss_function, cv=split)
scores = cross_validate(pipeline, Xmtr, Ymtr, cv=split, groups=groups, 
                        scoring=loss_function, return_train_score=True, n_jobs=-1,
                        cv=tqdm(split, total=groups.nunique()))
#grid.fit(Xmtr, Ymtr)
#estimator = grid.best_estimator_
#best_params = grid.best_params_
#Y_pred = estimator.predict(Xmte)
#results[name] = Y_pred.mean()
#results[name + '_params'] = best_params
results[name] = scores
print(f'model {name} score {scores}')

In [ ]:
# store results
with open(OUTPUT_FILE, 'wb') as o:
    pickle.dump(results, o)

## 03_ogo_cont_windows.ipynb

In [ ]:
def do_training(window_start = 0):
    columns = X.columns[window_start: window_start+WINDOW_LEN]

    X_window = X[columns]
    Y_window = Y[columns[0]].to_frame()
    X_window = utils.get_oh_matrix(X_window)
    X_window = utils.remove_useless_columns(X_window)
    #X_window['window_start'] = window_start
    
    if window_start == 0:
        logger.info(f'pos {window_start} compressed shape {X_window.shape}')
        logged = True

    split = LeaveOneGroupOut().split(X_window, Y_window, groups=groups)
    scores = cross_validate(model, X_window, Y_window, 
                            groups=groups,
                            n_jobs=N_JOBS, 
                            scoring=loss, 
                            verbose=VERBOSE, cv=split,
                            return_train_score=True, 
                            error_score=ERROR_SCORE)
    
    return generate_results(scores, window_start)

In [ ]:
limit = X.shape[1]-WINDOW_LEN
#limit = 2

partial_results = []
for i in range(limit):
    partial_results.append(do_training(window_start=i))

res_df = pd.concat(partial_results, ignore_index=True)

INFO:papermill:pos 0 compressed shape (3801, 87)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ..............., score=(train=-34.544, test=-22.702) total time=   0.6s
[CV] END ..............., score=(train=-33.178, test=-33.647) total time=   0.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s


[CV] END ..............., score=(train=-33.540, test=-28.852) total time=   0.6s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.7s remaining:    0.0s


[CV] END ..............., score=(train=-33.574, test=-34.175) total time=   0.6s


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.4s remaining:    0.0s


[CV] END ..............., score=(train=-33.564, test=-38.710) total time=   0.6s
[CV] END ..............., score=(train=-33.491, test=-37.791) total time=   0.6s
[CV] END ..............., score=(train=-33.628, test=-12.919) total time=   0.6s
[CV] END ..............., score=(train=-33.602, test=-15.691) total time=   0.6s
[CV] END ..............., score=(train=-33.610, test=-30.360) total time=   0.7s
[CV] END ................, score=(train=-33.569, test=-1.730) total time=   0.8s
[CV] END ..............., score=(train=-33.617, test=-11.461) total time=   0.6s
[CV] END ..............., score=(train=-33.592, test=-44.451) total time=   0.7s
[CV] END ................, score=(train=-33.614, test=-9.368) total time=   0.6s
[CV] END ................, score=(train=-33.620, test=-6.350) total time=   0.6s
[CV] END ................, score=(train=-33.630, test=-6.050) total time=   0.6s
[CV] END ..............., score=(train=-33.592, test=-42.197) total time=   0.6s
[CV] END ..............., sc